In [26]:
!pip install -Uqq fastbook


In [1]:
import fastbook

In [2]:
fastbook.setup_book()

In [3]:
from fastbook import *
from fastai.vision.widgets import *

In [4]:
from ddgs import DDGS #DuckDuckGo has changed the api so we need to update 
from fastcore.all import *

def search_images(keywords, max_images=200): 
    return L(DDGS().images(keywords, max_results=max_images)).itemgot('image')


In [5]:
img = search_images('grizzly bear')

len(img)

99

In [6]:
dest = 'images/grizzly.jpg'
download_url(img[0], dest, show_progress=False)

Path('images/grizzly.jpg')

In [ ]:
im = Image.open(dest)
im.to_thumb(128,128)

In [8]:
bear_type = 'grizzly', 'black', 'teddy'
path = Path('bears')

In [14]:
if not path.exists():
    path.mkdir()
    for o in bear_type:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images(f'{o} bear')
        download_images(dest, urls=results)
        time.sleep(5)

In [15]:
fns = get_image_files(path)
fns

(#266) [Path('bears/black/054ff5c1-710a-41b0-b964-d5f656d3c26a.jpg'),Path('bears/black/09873d99-1e22-449d-94ec-dd9ed85425d2.jpg'),Path('bears/black/0a32f2e2-32aa-4a8c-a2b0-a75836c9de8c.jpg'),Path('bears/black/0b2e843b-aa55-467d-8aa9-e3b87a6860a4.jpeg'),Path('bears/black/0d87c7f2-728d-4ed2-a7c2-be7312170a4a.jpg'),Path('bears/black/0f5f0f63-2bec-4f4e-a247-f8829f76707c.jpg'),Path('bears/black/0f8ce4a0-7443-4862-95b9-20660a0acf27.jpg'),Path('bears/black/121b1114-9a75-463d-bc3e-a87e40c34197.jpg'),Path('bears/black/1340a43f-f11c-4314-b8f7-076f3459a523.jpg'),Path('bears/black/14ecf3d8-25e0-4c6d-89ab-fe83c69a0c76.jpg'),Path('bears/black/156054ab-b30c-4e31-ba36-7149043a7729.jpg'),Path('bears/black/191746d9-c155-4f2f-afed-8ef86972ec28.jpg'),Path('bears/black/196671cf-8a57-4d70-bc48-767477754796.jpg'),Path('bears/black/1c88533c-e9d6-4dc7-8e46-e45ab83e3dcf.jpg'),Path('bears/black/230bb250-4808-41c0-96db-bf3b39b38110.jpg'),Path('bears/black/25c21572-701d-4677-945b-8f5b7b5a9258.jpg'),Path('bears/bla

In [16]:
failed = verify_images(fns)
failed

(#2) [Path('bears/black/df8bd153-8320-4d48-906e-b392a0a702a8.jpg'),Path('bears/grizzly/62246a9e-dba7-4bd9-a86e-e2f40b86366f.png')]

In [17]:
failed.map(Path.unlink)

(#2) [None,None]

In [ ]:
bears = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128)
)

dls = bears.dataloaders(path)

dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
bears = bears.new(item_tfms=Resize(128, ResizeMethod.Squish))
dls = bears.dataloaders(path)

dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
bears = bears.new(item_tfms=Resize(128, ResizeMethod.Pad, pad_mode=PadMode.Zeros))
dls = bears.dataloaders(path)

dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
bears = bears.new(item_tfms=RandomResizedCrop(128, min_scale=0.3))
dls = bears.dataloaders(path)

dls.train.show_batch(max_n=4, nrows=1, unique=True)

In [ ]:
bears = bears.new(item_tfms=Resize(128), batch_tfms=aug_transforms(mult=2))
dls = bears.dataloaders(path)

dls.train.show_batch(max_n=8, nrows=2, unique=True)

In [23]:
bears = bears.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = bears.dataloaders(path)

In [24]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

epoch,train_loss,valid_loss,error_rate,time
0,2.028474,1.123964,0.365385,00:29


C:\Users\VeeneetKumar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\PIL\Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


epoch,train_loss,valid_loss,error_rate,time
0,0.366931,0.164470,0.038462,00:38
1,0.291606,0.010970,0.000000,00:34
2,0.222858,0.003609,0.000000,00:33
3,0.176675,0.003150,0.000000,00:18


In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(5, nrows=1, figsize=(17,4))

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner

In [29]:
for idx in cleaner.delete(): cleaner.fns[idx].unlink()
for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)

In [30]:
learn.export('bear_classifier.pkl')